In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet50
from torchvision.datasets import VOCDetection, Caltech256, CIFAR10
from torch.utils.data import DataLoader
from PIL import Image
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
import numpy as np
from torcheval.metrics.functional import multiclass_f1_score, multiclass_accuracy

In [2]:
import os, sys
from os import listdir
from os.path import isfile, isdir, join
import cv2
import pickle as pkl
from typing import Dict, Iterable, Callable

In [3]:
torch.manual_seed(12345)

In [4]:
class ModifiedCIFAR10DatasetTe(CIFAR10):
    def __init__(self, path='./data', train = False, download_flag=True, validation_flag=True):
        super(ModifiedCIFAR10DatasetTe, self).__init__(path, train, None, None, download_flag)
        self.classes = []
        self.labels = []
        # self.means = torch.zeros((3, 1), dtype=torch.float32)
        # self.vars = torch.zeros((3, 1), dtype=torch.float32)
        self.means = torch.from_numpy(np.array([[0.5520],
                                    [0.5336],
                                    [0.5050]], dtype=np.float32))
        self.std_dev = torch.from_numpy(np.array([[0.1973],
                                    [0.1930],
                                    [0.2098]], dtype=np.float32))
        self.transform_mod = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
            # transforms.CenterCrop(224)
        ])
        self.normalize_tensor = transforms.Normalize(self.means, self.std_dev)

    def __getitem__(self, index):
        img, target = super(ModifiedCIFAR10DatasetTe, self).__getitem__(index)
        img_tensor = self.transform_mod(img)
        if img_tensor.dim()==2:
            img_tensor = img_tensor[None, ...].expand(3, -1, -1)
        elif img_tensor.shape[0]==1:
            img_tensor = img_tensor.expand(3, -1, -1)
        self.labels.append(target)
        img_tensor = transforms.Normalize(mean=[0.5520, 0.5336, 0.5050], std=[0.1973, 0.1930, 0.2098])(img_tensor)
        return img_tensor, target, index

In [5]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [6]:
datasetTe = ModifiedCIFAR10DatasetTe('./data', download_flag=True)
test_loader = DataLoader(datasetTe, batch_size=128, shuffle=False, num_workers=8)

Files already downloaded and verified


In [7]:
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, L1FMNAttack, L2PGD
import pandas as pd
import random
import matplotlib.pyplot as plt

In [8]:
indices = [1121, 1673, 2422,  3198,  4026,  43,    4697,  5157,  5391,  6000,  6461,  7094, 7752,  8516,  8873,  9308,
1150,  1732,  2650,  3331,  403,   4496,  4741,  5187,  5531,  6025,  6624,  7272,  8010,  8640,  9020,  95,
1166,  1907,  2831,  3374,  4066,  4497,  4933,  519,   5670,  6034,  6818,  7446,  819,   8661,  9083,  9618,
1221,  2088,  2868,  3400,  4167,  4544,  4952,  5233,  5685,  6329,  6837,  7566,  8237,  8663,  914,   9907,
1605,  2189,  2889,  3541,  4359,  4593,  5112,  5351,  5905,  6394,  6888,  7633,  8391,  8671,  9159,  9926,
1625,  2346,  3093,  3754,  4362,  4686,  5153,  5384,  5931,  6406,  703,   7644,  8451,  8708,  9219,  9980]

In [9]:
datasetTe = torch.utils.data.Subset(datasetTe, indices)
attack_dl = DataLoader(datasetTe, batch_size=32, shuffle=False)
print(len(attack_dl))

3


In [10]:
def tensor_to_img(inp, shift=False, normalize=False, scale:tuple=None):
    img = inp.cpu().squeeze().permute(1, 2, 0).numpy()
    if scale is not None:
        img[..., 0] = img[..., 0]*scale[1][0]
        img[..., 1] = img[..., 1]*scale[1][1]
        img[..., 2] = img[..., 2]*scale[1][2]
        img[..., 0] = img[..., 0]+scale[0][0]
        img[..., 1] = img[..., 1]+scale[0][1]
        img[..., 2] = img[..., 2]+scale[0][2]
    if shift:
        img[..., 0] = img[..., 0]-img[..., 0].min()
        img[..., 1] = img[..., 1]-img[..., 1].min()
        img[..., 2] = img[..., 2]-img[..., 2].min()
    if normalize:
        img[..., 0] = img[..., 0]/img[..., 0].max()
        img[..., 1] = img[..., 1]/img[..., 1].max()
        img[..., 2] = img[..., 2]/img[..., 2].max()
    img = np.clip(img*255., 0, 255).astype(np.uint8)
    return img

def norm_tensor(img, scale):
    img[..., 0] = img[..., 0]*scale[1][0]
    img[..., 1] = img[..., 1]*scale[1][1]
    img[..., 2] = img[..., 2]*scale[1][2]
    img[..., 0] = img[..., 0]+scale[0][0]
    img[..., 1] = img[..., 1]+scale[0][1]
    img[..., 2] = img[..., 2]+scale[0][2]
    return img

mean=np.array([0.4913997551666284, 0.48215855929893703, 0.4465309133731618])
std_dev=np.array([0.24703225141799082, 0.24348516474564, 0.26158783926049628])

In [11]:
%%time 
model = resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)
model.load_state_dict(torch.load('./models/sc_resnet50_voc2012_40.pth', map_location='cuda:0')['state_dict'])
model.eval()
model = model.to(device)
fmodel = PyTorchModel(model, bounds=(-10, 10), device=device)

clean_acc_avg = 0
numSamplesTotal = 0
robust_acc_avg = 0
# plt.figure(figsize=(18, 18))
save_dir = './sc_adversary'
if not isdir(save_dir):
    os.mkdir(save_dir)

attack_dict = {}
for batch_iter, (inputs, classes, idx) in enumerate(tqdm(attack_dl)):
    sampless = (inputs.to(device), classes.to(device))
    images, labels = ep.astensors(*sampless)
    # plt.imshow(images[0].raw.cpu().permute(1, 2, 0).numpy())
    # print(images.shape, labels.shape)
    numSamples = int(labels.shape[0])
    clean_acc = accuracy(fmodel, images, labels)
    attack = L2PGD()
    epsilons = [
        25, # - upper bound with 25
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    # print(raw_advs[0][0].raw.shape)
    # print(success[0].shape)
    # rimg = tensor_to_img(raw_advs[0][0].raw, False, False, (mean, std_dev))
    # cimg = tensor_to_img(clipped_advs[0][0].raw, False, False, (mean, std_dev))
    # plt.imshow(rimg)
    # plt.imshow(cimg)
    print(success[0])

    adversarial_noise = raw_advs[0].raw.cpu()-inputs.cpu()
    # print(adversarial_noise.shape)
    for b_id in range(inputs.shape[0]):
        adversarial_image = tensor_to_img(raw_advs[0].raw.cpu()[b_id], False, False, (mean, std_dev))
        adversarial_noise_image = tensor_to_img(adversarial_noise[b_id], False, False)
        # plt.imshow(adversarial_noise_image)
        cv2.imwrite(join(save_dir, '{}.png'.format(idx[b_id])), adversarial_image[..., ::-1])
        cv2.imwrite(join(save_dir, '{}_noise.png'.format(idx[b_id])), adversarial_noise_image[..., ::-1])
        attack_dict[idx[b_id].item()] = {'l2':torch.linalg.matrix_norm(torch.mean(norm_tensor(adversarial_noise[b_id],
                                                                                             (mean, std_dev)), axis=0), ord=2).item(), 
                                  'avg':np.mean(adversarial_noise_image)}
        # break
    
    robust_accuracy = np.count_nonzero(success[0])/success[0].shape[0]
    print('Robust Accuracy:{}| Clean Accuracy:{}'.format(robust_accuracy, clean_acc))
    clean_acc_avg += clean_acc
    robust_acc_avg += robust_accuracy
    # break
clean_acc_avg /= len(attack_dl)
robust_acc_avg /= len(attack_dl)
print(robust_acc_avg)
print(clean_acc_avg)
# df = pd.DataFrame(robust_acc_avg_np)
# df.to_csv("robust_accuracy_range_SC.csv",index=False)
with open('./sc_adversary.pkl', 'wb') as fp:
    pkl.dump(attack_dict, fp)

/home/ragraw06/UBCO/rishabh/AI_conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ragraw06/UBCO/rishabh/AI_conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


  0%|          | 0/3 [00:00<?, ?it/s]

/home/ragraw06/UBCO/rishabh/AI_conda/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


PyTorchTensor(tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
                       True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
                       True,  True,  True,  True,  True,  True,  True, False,  True,  True,
                       True,  True], device='cuda:1'))
Robust Accuracy:0.96875| Clean Accuracy:0.875
PyTorchTensor(tensor([True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True], device='cuda:1'))
Robust Accuracy:1.0| Clean Accuracy:0.875
PyTorchTensor(tensor([True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True], device='cuda:1'))
Robust Accuracy:1.0| Clean 

In [12]:
%%time 
model = resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)
model.load_state_dict(torch.load('./models/ft_resnet50_voc2012.pth', map_location='cuda:0')['state_dict'])
model.eval()
model = model.to(device)
fmodel = PyTorchModel(model, bounds=(-10, 10), device=device)

clean_acc_avg = 0
numSamplesTotal = 0
robust_acc_avg = 0
# plt.figure(figsize=(18, 18))
save_dir = './ft_adversary'
if not isdir(save_dir):
    os.mkdir(save_dir)

attack_dict = {}
for batch_iter, (inputs, classes, idx) in enumerate(tqdm(attack_dl)):
    sampless = (inputs.to(device), classes.to(device))
    images, labels = ep.astensors(*sampless)
    # plt.imshow(images[0].raw.cpu().permute(1, 2, 0).numpy())
    # print(images.shape, labels.shape)
    numSamples = int(labels.shape[0])
    clean_acc = accuracy(fmodel, images, labels)
    attack = L2PGD()
    epsilons = [
        25, # - upper bound with 25
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    # print(raw_advs[0][0].raw.shape)
    # print(success[0].shape)
    # rimg = tensor_to_img(raw_advs[0][0].raw, False, False, (mean, std_dev))
    # cimg = tensor_to_img(clipped_advs[0][0].raw, False, False, (mean, std_dev))
    # plt.imshow(rimg)
    # plt.imshow(cimg)
    print(success[0])

    adversarial_noise = raw_advs[0].raw.cpu()-inputs.cpu()
    for b_id in range(inputs.shape[0]):
        adversarial_image = tensor_to_img(raw_advs[0].raw.cpu()[b_id], False, False, (mean, std_dev))
        adversarial_noise_image = tensor_to_img(adversarial_noise[b_id], False, False)
        # plt.imshow(adversarial_noise_image)
        cv2.imwrite(join(save_dir, '{}.png'.format(idx[b_id])), adversarial_image[..., ::-1])
        cv2.imwrite(join(save_dir, '{}_noise.png'.format(idx[b_id])), adversarial_noise_image[..., ::-1])
        attack_dict[idx[b_id].item()] = {'l2':torch.linalg.matrix_norm(torch.mean(norm_tensor(adversarial_noise[b_id],
                                                                                             (mean, std_dev)), axis=0), ord=2).item(), 
                                  'avg':np.mean(adversarial_noise_image)}
        # break
    
    robust_accuracy = np.count_nonzero(success[0])/success[0].shape[0]
    print('Robust Accuracy:{}| Clean Accuracy:{}'.format(robust_accuracy, clean_acc))
    clean_acc_avg += clean_acc
    robust_acc_avg += robust_accuracy
    # break
clean_acc_avg /= len(attack_dl)
robust_acc_avg /= len(attack_dl)
print(robust_acc_avg)
print(clean_acc_avg)
# df = pd.DataFrame(robust_acc_avg_np)
# df.to_csv("robust_accuracy_range_FT.csv",index=False)
with open('./ft_adversary.pkl', 'wb') as fp:
    pkl.dump(attack_dict, fp)

  0%|          | 0/3 [00:00<?, ?it/s]

PyTorchTensor(tensor([True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True], device='cuda:1'))
Robust Accuracy:1.0| Clean Accuracy:0.875
PyTorchTensor(tensor([True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True], device='cuda:1'))
Robust Accuracy:1.0| Clean Accuracy:1.0
PyTorchTensor(tensor([True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True, True, True, True, True,
                      True, True, True, True, True, True, True, True], device='cuda:1'))
Robust Accuracy:1.0| Clean Accuracy:0.9375
1.0
0.9375
CPU times: user 55.9 s, sys: 4.75

In [13]:
class FeatureExtractor(nn.Module):
    def __init__(self, model: nn.Module, layers: Iterable[str]):
        super().__init__()
        self.model = model
        self.layers = layers
        self._features = {layer: torch.empty(0) for layer in layers}

        for layer_id in layers:
            layer = dict([*self.model.named_modules()])[layer_id]
            layer.register_forward_hook(self.save_outputs_hook(layer_id))

    def save_outputs_hook(self, layer_id: str) -> Callable:
        def fn(_, input, output):
            self._features[layer_id] = output
        return fn

    def clear_features(self):
        self._features = {layer: torch.empty(0) for layer in layers}

    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        _ = self.model(x)
        return self._features

In [14]:
model = resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)
model.load_state_dict(torch.load('./models/sc_resnet50_voc2012_40.pth', map_location='cuda:0')['state_dict'])
model.eval()
model = model.to(device)
fmodel = PyTorchModel(model, bounds=(-10, 10), device=device)

conv_layers = []
for name, layer in model.named_modules():
    if 'conv' in name and isinstance(layer, nn.Conv2d):
        conv_layers.append(name)
print(conv_layers)

extractor = FeatureExtractor(model, conv_layers)
extractor.eval()
extractor.to(device)

['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.0.conv3', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.1.conv3', 'layer1.2.conv1', 'layer1.2.conv2', 'layer1.2.conv3', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.conv3', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.1.conv3', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.2.conv3', 'layer2.3.conv1', 'layer2.3.conv2', 'layer2.3.conv3', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.conv3', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.1.conv3', 'layer3.2.conv1', 'layer3.2.conv2', 'layer3.2.conv3', 'layer3.3.conv1', 'layer3.3.conv2', 'layer3.3.conv3', 'layer3.4.conv1', 'layer3.4.conv2', 'layer3.4.conv3', 'layer3.5.conv1', 'layer3.5.conv2', 'layer3.5.conv3', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.conv3', 'layer4.1.conv1', 'layer4.1.conv2', 'layer4.1.conv3', 'layer4.2.conv1', 'layer4.2.conv2', 'layer4.2.conv3']


FeatureExtractor(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [15]:
save_dir = './sc_adversary_features'
if not isdir(save_dir):
    os.mkdir(save_dir)

attack_dict = {}
images_original_dir = join(save_dir, 'original')
images_attack_dir = join(save_dir, 'adversary')
if not isdir(images_original_dir):
    os.mkdir(images_original_dir)
if not isdir(images_attack_dir):
    os.mkdir(images_attack_dir)
    
for batch_iter, (inputs, classes, idx) in enumerate(tqdm(attack_dl)):
    sampless = (inputs.to(device), classes.to(device))
    images, labels = ep.astensors(*sampless)
    # plt.imshow(images[0].raw.cpu().permute(1, 2, 0).numpy())
    # print(images.shape, labels.shape)
    numSamples = int(labels.shape[0])
    clean_acc = accuracy(fmodel, images, labels)
    attack = L2PGD()
    epsilons = [
        25, # - upper bound with 25
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    
    with torch.no_grad():
        o_input = inputs.to(device)
        # for b_i in range(o_input.shape[0]):
        #     plt.imshow(tensor_to_img(o_input[b_i].cpu(), False, False, scale=(mean, std_dev)))
        #     plt.show()
        #     cv2.imwrite(join(images_original_dir, 'src', '{}.png'.format(data_dict['idx'][b_i])), 
        #                 tensor_to_img(o_input[b_i].cpu(), False, False, scale=(mean, std_dev))[..., ::-1])
        o_output = extractor(o_input).copy()
        
        p_input = raw_advs[0].raw.to(device)
        for b_i in range(p_input.shape[0]):
            attack_dict[idx[b_i].item()] = {}
        #     plt.imshow(tensor_to_img(p_input[b_i].cpu(), False, False, scale=(mean, std_dev)))
        #     plt.show()
        #     cv2.imwrite(join(images_attack_dir, 'src', '{}.png'.format(data_dict['idx'][b_i])), 
        #                 tensor_to_img(p_input[b_i].cpu(), False, False, scale=(mean, std_dev))[..., ::-1])
        p_output = extractor(p_input).copy()
        # # print([(k,o_output[k].shape) for k in o_output.keys()])
        # # print([(k,p_output[k].shape) for k in p_output.keys()])
        
        for k in tqdm(p_output.keys()):
            # print(p_output[k].shape)
            if not isdir(join(images_original_dir, k)):
                os.mkdir(join(images_original_dir, k))
            if not isdir(join(images_attack_dir, k)):
                os.mkdir(join(images_attack_dir, k))
                
            for b_i in range(p_output[k].shape[0]):
                with open(join(images_original_dir, k, '{}.pkl'.format(idx[b_i])), 'wb') as fp:
                    pkl.dump(o_output[k][b_i].detach().cpu().numpy(), fp)            
                with open(join(images_attack_dir, k, '{}.pkl'.format(idx[b_i])), 'wb') as fp:
                    pkl.dump(p_output[k][b_i].detach().cpu().numpy(), fp)
                
                diff_norm = torch.linalg.matrix_norm(p_output[k][b_i]-o_output[k][b_i], ord=2)
                attack_dict[idx[b_i].item()][k]=np.sqrt(diff_norm.cpu().numpy())
        # break
with open('./sc_adversary_features.pkl', 'wb') as fp:
    pkl.dump(attack_dict, fp)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

In [16]:
model = resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)
model.load_state_dict(torch.load('./models/ft_resnet50_voc2012.pth', map_location='cuda:0')['state_dict'])
model.eval()
model = model.to(device)
fmodel = PyTorchModel(model, bounds=(-10, 10), device=device)

conv_layers = []
for name, layer in model.named_modules():
    if 'conv' in name and isinstance(layer, nn.Conv2d):
        conv_layers.append(name)
print(conv_layers)

extractor = FeatureExtractor(model, conv_layers)
extractor.eval()
extractor.to(device)

['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.0.conv3', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.1.conv3', 'layer1.2.conv1', 'layer1.2.conv2', 'layer1.2.conv3', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.conv3', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.1.conv3', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.2.conv3', 'layer2.3.conv1', 'layer2.3.conv2', 'layer2.3.conv3', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.conv3', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.1.conv3', 'layer3.2.conv1', 'layer3.2.conv2', 'layer3.2.conv3', 'layer3.3.conv1', 'layer3.3.conv2', 'layer3.3.conv3', 'layer3.4.conv1', 'layer3.4.conv2', 'layer3.4.conv3', 'layer3.5.conv1', 'layer3.5.conv2', 'layer3.5.conv3', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.conv3', 'layer4.1.conv1', 'layer4.1.conv2', 'layer4.1.conv3', 'layer4.2.conv1', 'layer4.2.conv2', 'layer4.2.conv3']


FeatureExtractor(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [17]:
save_dir = './ft_adversary_features'
if not isdir(save_dir):
    os.mkdir(save_dir)

attack_dict = {}
images_original_dir = join(save_dir, 'original')
images_attack_dir = join(save_dir, 'adversary')
if not isdir(images_original_dir):
    os.mkdir(images_original_dir)
if not isdir(images_attack_dir):
    os.mkdir(images_attack_dir)
    
for batch_iter, (inputs, classes, idx) in enumerate(tqdm(attack_dl)):
    sampless = (inputs.to(device), classes.to(device))
    images, labels = ep.astensors(*sampless)
    # plt.imshow(images[0].raw.cpu().permute(1, 2, 0).numpy())
    # print(images.shape, labels.shape)
    numSamples = int(labels.shape[0])
    clean_acc = accuracy(fmodel, images, labels)
    attack = L2PGD()
    epsilons = [
        25, # - upper bound with 25
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    
    with torch.no_grad():
        o_input = inputs.to(device)
        # for b_i in range(o_input.shape[0]):
        #     plt.imshow(tensor_to_img(o_input[b_i].cpu(), False, False, scale=(mean, std_dev)))
        #     plt.show()
        #     cv2.imwrite(join(images_original_dir, 'src', '{}.png'.format(data_dict['idx'][b_i])), 
        #                 tensor_to_img(o_input[b_i].cpu(), False, False, scale=(mean, std_dev))[..., ::-1])
        o_output = extractor(o_input).copy()
        
        p_input = raw_advs[0].raw.to(device)
        for b_i in range(p_input.shape[0]):
            attack_dict[idx[b_i].item()] = {}
        #     plt.imshow(tensor_to_img(p_input[b_i].cpu(), False, False, scale=(mean, std_dev)))
        #     plt.show()
        #     cv2.imwrite(join(images_attack_dir, 'src', '{}.png'.format(data_dict['idx'][b_i])), 
        #                 tensor_to_img(p_input[b_i].cpu(), False, False, scale=(mean, std_dev))[..., ::-1])
        p_output = extractor(p_input).copy()
        # # print([(k,o_output[k].shape) for k in o_output.keys()])
        # # print([(k,p_output[k].shape) for k in p_output.keys()])

        for k in tqdm(p_output.keys()):
            # print(p_output[k].shape)
            if not isdir(join(images_original_dir, k)):
                os.mkdir(join(images_original_dir, k))
            if not isdir(join(images_attack_dir, k)):
                os.mkdir(join(images_attack_dir, k))
                
            for b_i in range(p_output[k].shape[0]):
                with open(join(images_original_dir, k, '{}.pkl'.format(idx[b_i])), 'wb') as fp:
                    pkl.dump(o_output[k][b_i].detach().cpu().numpy(), fp)            
                with open(join(images_attack_dir, k, '{}.pkl'.format(idx[b_i])), 'wb') as fp:
                    pkl.dump(p_output[k][b_i].detach().cpu().numpy(), fp)
                
                diff_norm = torch.linalg.matrix_norm(p_output[k][b_i]-o_output[k][b_i], ord=2)
                attack_dict[idx[b_i].item()][k]=np.sqrt(diff_norm.cpu().numpy())
        # break
with open('./ft_adversary_features.pkl', 'wb') as fp:
    pkl.dump(attack_dict, fp)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]